# Topic Guidance

The idea behind this use case is to guide the model to stick to certain topics and avoid answering certain questions. In other words, the Guardrail layer examines every user input and filters them accordingly. 

In this notebook we will implement a simple example where the chatbot is instructed not to answer any sport-related questions. We will use both OpenAI and Llama2 models. 

In [ ]:
## Load environment 

import os
from dotenv import load_dotenv

load_dotenv()

# Guardrails with OpenAI

Load Guardrails configuration files located under `topic_rail_config/openai`.

In [11]:
from nemoguardrails import LLMRails, RailsConfig

## Load rails config
config = RailsConfig.from_path("./topic_rail_config/openai")

## Create rails
rails = LLMRails(config, verbose=True)

When passed a typical greeting, protective guardrails are not activated:

In [12]:
res = await rails.generate_async(prompt="Hey there!")
print(res)

Hi there! How can I help you?
How are you doing today?


When asked about a match result or a football player, the chatbot effectively evades the question as designed:

In [13]:
res = await rails.generate_async(prompt="Tell me about the latest soccer match.")
print(res)

I'm a shopping assistant, I don't like to talk of sports.
However, I can help you find the latest soccer merchandise or equipment. Is there anything specific you are looking for?


In [14]:
res = await rails.generate_async(prompt="Do you know who is Lionel Messi?")
print(res)

I'm a shopping assistant, I don't like to talk of sports.
However, I can help you with finding and purchasing any products you need.


# Guardrails with Llama2

Nemo Guardrails also supports loading models from HuggingFace hub. We will use this to load Llama2. 

Load the HuggingFace model and create a pipeline:

In [16]:
## Load hugging face credentials
import huggingface_hub
huggingface_hub.login(token=os.environ["HUGGINGFACEHUB_API_TOKEN"])

## Load model and tokenizer
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf")

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /Users/sofiaperez/.cache/huggingface/token
Login successful


Loading checkpoint shards: 100%|██████████| 2/2 [00:19<00:00,  9.90s/it]


In [ ]:
## Define pipeline
pipe = pipeline(
    "text-generation",
    model=model, 
    tokenizer=tokenizer,
    max_length=1024,
    do_sample=True,
    temperature=0.1,
    top_p=0.95,
    logprobs=None,
    top_k=40,
    repetition_penalty=1.1
)

Wrap the pipeline using Langchain HuggingFacePipeline class. Then wrap it again using Nemo’s get_llm_instance_wrapper function and register it using register_llm_provider.

In [18]:
llm = HuggingFacePipeline(pipeline=pipe)

from nemoguardrails.llm.helpers import get_llm_instance_wrapper
from nemoguardrails.llm.providers import register_llm_provider

HFPipeline = get_llm_instance_wrapper(
    llm_instance=llm, llm_type="hf_pipeline_llama2"
)

register_llm_provider("hf_pipeline_llama2", HFPipeline)

Load Guardrails configuration files located under `topic_rail_config/llama2`.

In [27]:
from nemoguardrails import LLMRails, RailsConfig

## Initialize rails config
config = RailsConfig.from_path("./topic_rail_config/llama2")

## Create rails
rails = LLMRails(config)

When passed a typical greeting, protective guardrails are not activated:

In [21]:
res = await rails.generate_async(prompt="hello")
print(res)

Parameter temperature does not exist for WrapperLLM


Hello! How can I assist you today?
How are you today?


When asked about a match result or a football player, the chatbot effectively evades the question as designed:

In [28]:
res = await rails.generate_async(prompt="Tell me about the latest soccer match.")
print(res)

Parameter temperature does not exist for WrapperLLM


I'm a shopping assistant, I don't like to talk of sports.
Is there anything else I can help you with?


In [29]:
res = await rails.generate_async(prompt="Do you know who is Lionel Messi?")
print(res)

Parameter temperature does not exist for WrapperLLM


I'm a shopping assistant, I don't like to talk of sports.
Is there anything else I can help you with?
